In [1]:
import org.apache.spark.sql.SparkSession

In [2]:
val spark = SparkSession
  .builder()
  .appName("lab5")
  .config("spark.executor.instances", "3")
  .getOrCreate()

spark = org.apache.spark.sql.SparkSession@466b9048


org.apache.spark.sql.SparkSession@466b9048

In [3]:
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark._

In [4]:
import spark.implicits._

In [6]:
val train_data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("labs/lab05_train.csv").repartition(9)
val test_data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("labs/lab05_test.csv").repartition(9)

train_data = [_c0: int, ID: int ... 115 more fields]
test_data = [_c0: int, ID: int ... 114 more fields]


lastException: Throwable = null


[_c0: int, ID: int ... 114 more fields]

In [8]:
val null_thresh = 320764*0.5               

val to_drop = train_data.columns.filter(
    c => train_data.agg(
        sum(when(train_data(c).isNull, 1).otherwise(0)).alias(c)
    ).first().getLong(0) > null_thresh
)

null_thresh = 160382.0
to_drop = Array(PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, CLNT_TRUST_RELATION, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, APP_MARITAL_STATUS, CNT_TRAN_MED_TENDENCY1M, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, APP_TRAVEL_PASS, CNT_ACCEPTS_MTP, APP_CAR, SUM_TRAN_CLO_TENDENCY1M, APP_POSITION_TYPE, CNT_TRAN_MED_TENDENCY3M, SUM_TRAN_MED_TENDENCY3M, APP_EMP_TYPE, SUM_TRAN_CLO_TENDENCY3M, LDEAL_TENOR_MAX, LDEAL_YQZ_CHRG, DEAL_YQZ_IR_MAX, LDEAL_YQZ...


Array(PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, CLNT_TRUST_RELATION, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, APP_MARITAL_STATUS, CNT_TRAN_MED_TENDENCY1M, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, APP_TRAVEL_PASS, CNT_ACCEPTS_MTP, APP_CAR, SUM_TRAN_CLO_TENDENCY1M, APP_POSITION_TYPE, CNT_TRAN_MED_TENDENCY3M, SUM_TRAN_MED_TENDENCY3M, APP_EMP_TYPE, SUM_TRAN_CLO_TENDENCY3M, LDEAL_TENOR_MAX, LDEAL_YQZ_CHRG, DEAL_YQZ_IR_MAX, LDEAL_YQZ...

In [9]:
val train_cleaned = train_data.drop(to_drop : _*).drop("_c0").where(col("TARGET").isNotNull)
val test_cleaned = test_data.drop(to_drop : _*).drop("_c0")

train_cleaned = [ID: int, CR_PROD_CNT_IL: int ... 49 more fields]
test_cleaned = [ID: int, CR_PROD_CNT_IL: int ... 48 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 48 more fields]

In [10]:
val numeric_columns = train_cleaned.schema.fields.filter(x => x.dataType == DoubleType).map(x => x.name)

numeric_columns = Array(AMOUNT_RUB_CLO_PRC, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_SAVE_3M, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, TRANS_COUNT_NAS_PRC, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, REST_DYNAMIC_CUR_1M, REST_AVG_PAYM, LDEAL_GRACE_DAYS_PCT_MED, REST_DYNAMIC_CUR_3M, CNT_TRAN_SUP_TENDENCY3M, TURNOVER_DYNAMIC_CUR_1M, REST_DYNAMIC_PAYM_3M, SUM_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_IL_3M, CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_ATM_TENDENCY1M, TURNOVER_DYNAMIC_IL_3M, SUM_TRAN_ATM_TENDENCY3M, SUM_TRAN_ATM_TENDENCY1M, REST_DYNAMIC_PAYM_1M, TURNOVER_DYNAMIC_CUR_3M, CLNT_SETUP_TENOR, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_PAYM_1M, TRANS_AMOUNT_TENDENCY3...


Array(AMOUNT_RUB_CLO_PRC, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_SAVE_3M, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, TRANS_COUNT_NAS_PRC, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, REST_DYNAMIC_CUR_1M, REST_AVG_PAYM, LDEAL_GRACE_DAYS_PCT_MED, REST_DYNAMIC_CUR_3M, CNT_TRAN_SUP_TENDENCY3M, TURNOVER_DYNAMIC_CUR_1M, REST_DYNAMIC_PAYM_3M, SUM_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_IL_3M, CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_ATM_TENDENCY1M, TURNOVER_DYNAMIC_IL_3M, SUM_TRAN_ATM_TENDENCY3M, SUM_TRAN_ATM_TENDENCY1M, REST_DYNAMIC_PAYM_1M, TURNOVER_DYNAMIC_CUR_3M, CLNT_SETUP_TENOR, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_PAYM_1M, TRANS_AMOUNT_TENDENCY3...

In [11]:
val imputer_output = numeric_columns.map(x => x.concat("_imputed"))

imputer_output = Array(AMOUNT_RUB_CLO_PRC_imputed, TURNOVER_DYNAMIC_IL_1M_imputed, AMOUNT_RUB_SUP_PRC_imputed, REST_DYNAMIC_FDEP_1M_imputed, REST_DYNAMIC_SAVE_3M_imputed, REST_AVG_CUR_imputed, AMOUNT_RUB_NAS_PRC_imputed, TRANS_COUNT_SUP_PRC_imputed, TRANS_COUNT_NAS_PRC_imputed, TURNOVER_CC_imputed, TRANS_COUNT_ATM_PRC_imputed, AMOUNT_RUB_ATM_PRC_imputed, TURNOVER_PAYM_imputed, REST_DYNAMIC_FDEP_3M_imputed, REST_DYNAMIC_IL_1M_imputed, REST_DYNAMIC_CUR_1M_imputed, REST_AVG_PAYM_imputed, LDEAL_GRACE_DAYS_PCT_MED_imputed, REST_DYNAMIC_CUR_3M_imputed, CNT_TRAN_SUP_TENDENCY3M_imputed, TURNOVER_DYNAMIC_CUR_1M_imputed, REST_DYNAMIC_PAYM_3M_imputed, SUM_TRAN_SUP_TENDENCY3M_imputed, REST_DYNAMIC_IL_3M_imputed, CNT_TRAN_ATM_TENDENCY3M_imputed, CNT_TRAN_ATM_TENDENCY1M_imputed, TURNOV...


Array(AMOUNT_RUB_CLO_PRC_imputed, TURNOVER_DYNAMIC_IL_1M_imputed, AMOUNT_RUB_SUP_PRC_imputed, REST_DYNAMIC_FDEP_1M_imputed, REST_DYNAMIC_SAVE_3M_imputed, REST_AVG_CUR_imputed, AMOUNT_RUB_NAS_PRC_imputed, TRANS_COUNT_SUP_PRC_imputed, TRANS_COUNT_NAS_PRC_imputed, TURNOVER_CC_imputed, TRANS_COUNT_ATM_PRC_imputed, AMOUNT_RUB_ATM_PRC_imputed, TURNOVER_PAYM_imputed, REST_DYNAMIC_FDEP_3M_imputed, REST_DYNAMIC_IL_1M_imputed, REST_DYNAMIC_CUR_1M_imputed, REST_AVG_PAYM_imputed, LDEAL_GRACE_DAYS_PCT_MED_imputed, REST_DYNAMIC_CUR_3M_imputed, CNT_TRAN_SUP_TENDENCY3M_imputed, TURNOVER_DYNAMIC_CUR_1M_imputed, REST_DYNAMIC_PAYM_3M_imputed, SUM_TRAN_SUP_TENDENCY3M_imputed, REST_DYNAMIC_IL_3M_imputed, CNT_TRAN_ATM_TENDENCY3M_imputed, CNT_TRAN_ATM_TENDENCY1M_imputed, TURNOV...

In [12]:
import org.apache.spark.ml.feature.{Imputer, VectorAssembler}
import org.apache.spark.ml.Pipeline

In [13]:
val imputer = new Imputer().setInputCols(numeric_columns).setOutputCols(imputer_output)
val assembler = new VectorAssembler().setInputCols(imputer.getOutputCols).setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(imputer, assembler))
val fitted_pipeline = pipeline.fit(train_cleaned)

imputer = imputer_e3fa95e85c63
assembler = vecAssembler_cba2039f09b1
pipeline = pipeline_e8f05922ee11
fitted_pipeline = pipeline_e8f05922ee11


pipeline_e8f05922ee11

In [14]:
val train_features = fitted_pipeline.transform(train_cleaned).select("ID", "features", "TARGET")
val test_features = fitted_pipeline.transform(test_cleaned).select("ID", "features")

train_features = [ID: int, features: vector ... 1 more field]
test_features = [ID: int, features: vector]


[ID: int, features: vector]

In [15]:
import org.apache.spark.ml.regression.GBTRegressor

In [16]:
val gbt = new GBTRegressor()
  .setLabelCol("TARGET")
  .setFeaturesCol("features")
  .setMaxIter(10)

gbt = gbtr_4cecd33ca34b


gbtr_4cecd33ca34b

In [17]:
val gbtModel = gbt.fit(train_features)

gbtModel = GBTRegressionModel (uid=gbtr_4cecd33ca34b) with 10 trees


GBTRegressionModel (uid=gbtr_4cecd33ca34b) with 10 trees

In [18]:
val predictions = gbtModel.transform(test_features)

predictions = [ID: int, features: vector ... 1 more field]


[ID: int, features: vector ... 1 more field]

In [20]:
val formatted_predictions = predictions.select(col("ID").alias("id"), col("prediction").alias("target"))

formatted_predictions = [id: int, target: double]


[id: int, target: double]

In [21]:
formatted_predictions.show

+------+--------------------+
|    id|              target|
+------+--------------------+
|442725|  0.1591927402462529|
|502622| 0.16120329311624923|
|569167|0.002736338430393...|
|230826|  0.1100732760530509|
|502658|0.002608095753557...|
|170369| 0.11693475623516146|
|459600|-0.00294007109008...|
|148340|-0.03186344348972775|
|269815|  0.4325763692168051|
|357879|0.011109810359935562|
|401047| 0.11161226830316882|
|530158| 0.16671147820044654|
|454695|0.002268132043352999|
|325590| 0.20106099976426414|
|264854|0.020857084734665313|
|334936|0.013040496333126515|
|579740| 0.10121339243497375|
|365034|  0.0122947736804144|
|532477| 0.03229644670599128|
|461677|0.016702041738913244|
+------+--------------------+
only showing top 20 rows



In [22]:
formatted_predictions.coalesce(1).write.mode(SaveMode.Overwrite).option("delimiter", "\t").option("header", "true").csv("lab05.csv")

In [38]:
sc.stop